In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [3]:
anon = pd.read_csv('C:/Users/Phuoc Le/Data_Anom/VinAnonyme_702', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [3]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [4]:
print(anon)

                                                         id  \
1         289b8475645ccc08b267c8ee1053172a3f2288bcf29086...   
2         cc69de8b681874f07678d2b0ab1a8512b899f610bcf541...   
3         2c99ff806998f29bcba014e6e1caeea59bd003e9a70557...   
4         731a45c31825349d7f2435cfc0e1b55e523edf076e6edd...   
5         e61ee967b9034e8c75ae55fc325db5055e48281b24dcc0...   
...                                                     ...   
34551842  705e21bd29a5562bd6184e79e0d27069874376d896fdf6...   
34551843  bba88f6a26823b046720a0647d5559caae31510cf45e7a...   
34551846  a4d65da444ea089bf5f7dd35d92d4fb18699acc0543669...   
34551847  e9eb88ff15b47db26645f42c1dbd1cd6b5e9f2a7eff812...   
34551848  bba88f6a26823b046720a0647d5559caae31510cf45e7a...   

                        date  latitude  longitude  week  
1        2015-04-30 06:47:00  4.588031  45.826524    18  
2        2015-05-02 13:14:00  6.912323  48.466316    18  
3        2015-04-16 07:53:00 -6.268315  53.349914    16  
4        20

In [5]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['...'] 449


In [6]:
#Set id to 'DEL' with lines duplicated and retain only the first occurrence of each unique combination of values in these columns.
id_time = df.drop_duplicates(['id', 'date', 'latitude', 'longitude'])
print(id_time.shape)

index_not_change = np.array(id_time.index)
df.loc[~df.index.isin(index_not_change) , 'id'] = "DEL"

print(df.head())

(16654892, 5)
    id                date  latitude  longitude  week
0    1 2015-03-04 00:35:00  4.870147  45.772140    10
1    1 2015-03-04 00:35:00  4.870218  45.772095    10
2    1 2015-03-04 00:35:00  4.870210  45.772072    10
3  DEL 2015-03-04 00:35:00  4.870210  45.772072    10
4  DEL 2015-03-04 00:35:00  4.870210  45.772072    10


In [7]:
new_df = df.loc[df['id'] != 'DEL']

In [8]:
for week, group in new_df.groupby('week'):
    print(f'Week {week}:')
    new_df_week = new_df[new_df['week'] == week]
    count_id_new_df_week = new_df_week['id'].value_counts().reset_index()
    count_id_new_df_week.columns = ['id', 'count']
    count_id_new_df_week = count_id_new_df_week.sort_values(by='count', ascending=True)
    list_count_id_new_df_week = count_id_new_df_week['id'].tolist()
    print(list_count_id_new_df_week, len(list_count_id_new_df_week))
    print()

Week 10:
[98, 18, 63, 107, 66, 5, 78, 32, 30, 73, 29, 77, 16, 68, 83, 60, 35, 48, 13, 67, 87, 28, 2, 34, 62, 49, 17, 110, 39, 44, 25, 26, 71, 41, 54, 24, 43, 1, 59, 23, 21, 31, 89, 58, 37, 15, 55, 51, 72, 69, 75, 9, 27, 14, 52, 50, 7, 36, 8, 42, 38, 11, 4, 6] 64

Week 11:
[29, 98, 5, 58, 16, 48, 77, 30, 21, 107, 78, 110, 49, 63, 54, 68, 37, 60, 25, 75, 41, 13, 36, 1, 34, 23, 83, 6, 62, 73, 67, 89, 14, 52, 15, 87, 24, 11, 44, 2, 31, 17, 59, 26, 7, 43, 9, 55, 28, 8, 72, 53, 38, 42, 4, 51, 27, 69, 50] 59

Week 12:
[34, 63, 84, 38, 16, 65, 71, 55, 78, 44, 68, 70, 15, 77, 9, 62, 53, 1, 87, 18, 30, 66, 107, 24, 28, 32, 41, 81, 73, 52, 31, 60, 49, 43, 8, 67, 75, 6, 26, 51, 42, 2, 69, 59, 72, 13, 17, 50, 27, 7, 4] 51

Week 13:
[44, 71, 29, 70, 77, 43, 54, 9, 84, 41, 63, 49, 18, 83, 32, 35, 68, 60, 52, 24, 30, 78, 17, 31, 66, 65, 28, 51, 75, 1, 62, 67, 13, 59, 73, 8, 53, 72, 26, 7, 2, 42, 27, 69, 50, 81, 4, 6] 48

Week 14:
[71, 52, 17, 84, 54, 29, 27, 35, 9, 78, 68, 70, 53, 32, 63, 28, 49, 73, 

In [12]:
dict_id_ori = {
    str("2015-10"): [
        98, 18, 63, 107, 66, 5, 78, 32, 30, 73, 29, 77, 16, 68, 83, 60, 35, 48, 13, 67, 87, 28, 2, 34, 62, 49, 17, 110, 39, 44, 25, 26, 71, 41, 54, 24, 43, 1, 59, 23, 21, 31, 89, 58, 37, 15, 55, 51, 72, 69, 75, 9, 27, 14, 52, 50, 7, 36, 8, 42, 38, 11, 4, 6
    ],
    str("2015-11"): [
        29, 98, 5, 58, 16, 48, 77, 30, 21, 107, 78, 110, 49, 63, 54, 68, 37, 60, 25, 75, 41, 13, 36, 1, 34, 23, 83, 6, 62, 73, 67, 89, 14, 52, 15, 87, 24, 11, 44, 2, 31, 17, 59, 26, 7, 43, 9, 55, 28, 8, 72, 53, 38, 42, 4, 51, 27, 69, 50
    ], 
    str("2015-12"): [
        34, 63, 84, 38, 16, 65, 71, 55, 78, 44, 68, 70, 15, 77, 9, 62, 53, 1, 87, 18, 30, 66, 107, 24, 28, 32, 41, 81, 73, 52, 31, 60, 49, 43, 8, 67, 75, 6, 26, 51, 42, 2, 69, 59, 72, 13, 17, 50, 27, 7, 4
    ], 
    str("2015-13"): [
        44, 71, 29, 70, 77, 43, 54, 9, 84, 41, 63, 49, 18, 83, 32, 35, 68, 60, 52, 24, 30, 78, 17, 31, 66, 65, 28, 51, 75, 1, 62, 67, 13, 59, 73, 8, 53, 72, 26, 7, 2, 42, 27, 69, 50, 81, 4, 6
    ], 
    str("2015-14"): [
        71, 52, 17, 84, 54, 29, 27, 35, 9, 78, 68, 70, 53, 32, 63, 28, 49, 73, 66, 75, 30, 24, 41, 67, 81, 62, 51, 83, 31, 1, 72, 59, 13, 42, 7, 69, 2, 65, 8, 4, 50, 6
    ], 
    str("2015-15"): [
        26, 27, 71, 44, 83, 63, 65, 18, 49, 68, 32, 39, 66, 35, 30, 31, 13, 7, 51, 41, 24, 75, 73, 62, 67, 69, 42, 2, 50, 59, 9, 1, 72, 8, 81, 4, 6
    ], 
    str("2015-16"): [
        83, 87, 31, 66, 41, 49, 32, 7, 78, 36, 35, 48, 26, 68, 30, 63, 13, 39, 71, 18, 9, 27, 73, 24, 8, 51, 50, 75, 67, 62, 42, 59, 72, 2, 1, 69, 4, 6
    ], 
    str("2015-17"): [
        87, 83, 78, 77, 71, 26, 51, 66, 18, 63, 41, 30, 8, 27, 17, 32, 68, 73, 75, 1, 24, 13, 50, 59, 67, 2, 42, 69, 7, 72, 4
    ], 
    str("2015-18"): [
        36, 39, 32, 18, 63, 30, 89, 17, 66, 77, 78, 71, 68, 83, 73, 87, 26, 50, 75, 24, 51, 59, 67, 1, 62, 42, 69, 72, 2, 4
    ], 
    str("2015-19"): [
        58, 17, 32, 43, 13, 63, 73, 66, 26, 83, 30, 87, 39, 18, 51, 9, 68, 24, 1, 62, 59, 75, 67, 60, 42, 50, 2, 72, 69, 4
    ], 
    str("2015-20"): [
        13, 77, 18, 2, 66, 73, 51, 68, 42, 75, 1, 24, 67, 72, 62, 59, 50, 69, 4
    ]
}

In [9]:
for week, group in anon.groupby('week'):
    print(f'Week {week}:')
    anon_week = anon[anon['week'] == week]
    count_id_anon_week = anon_week['id'].value_counts().reset_index()
    count_id_anon_week.columns = ['id', 'count']
    count_id_anon_week = count_id_anon_week.sort_values(by='count', ascending=True)
    list_count_id_anon_week = count_id_anon_week['id'].tolist()
    print(list_count_id_anon_week, len(list_count_id_anon_week))
    print()

Week 10:
['848d371053f5c2eed1c84b443e4e3a6b9bb50ee3a949cd80ba50b8495d6f3d6d', 'c83ec907476a3101632c878366027ed03068d75db8accbb50675a818bae944a6', '0f27aa12029fd459c6fe3195eb801ca37acda08753ebdc9ca5a2ab7c1e791706', '3fcd9e6d2295cb93c852b9af95f8fb0fb7dc242b9557a0df45af33ecf9107c56', 'c60a6e26f74c9be0cd8b3f5fcdf8e20ae2e4c2efd20f3ec78b6b9b782665f201', '64073b3845fed8476608b0ab5a9ab52c5869a6cdbaad5c5f0faa93467d0ef6ba', 'a5e1589dc872f333508a84f551ec37aa3b701080fb47dc3ce1eb9c7442bd0599', '386d7405944c35803554176ec361cdd789920d1b1d538b759438f96acc9f58f6', '7741a739b8d6bedb5ad20c247f16f4b3842f75b5e755130f6e134e5edac9036c', '779150f54f4d27efec1b1a0fe1a84ad8f6502b8f8e1fe145d4fc8c614dc9364b', 'ec39639428c541c343da55fd9763fac38251e53316bc2926e92dd16ba85d196e', '79d1f4d0b438aa92efa4d60b0f12bac6d65d3632f4da59b25381376391726933', '4f7c38079e19d8622303e7891d4d4e34a5a3ecefa556676af4b28f1ea4ed4284', 'e77832d6d6260526c9047e410736ec0d93c9a7ae38184b08e13070d135d6b938', 'd183211bee220bb2430d38199362e7ff096af

['3cd165eee7b041da6f88ed3685dd1f3793b3c5cb87e523310b58711983f3f754', 'e8b7711943eb868f29cc108e55627666e3d704b1802090f271d5d0dbe6689a84', 'f0c5ee05ff0751db09b41dfc1ccabc3dfab86d2c579bee76473938da511a12fe', '09ff6fde378760ad033445232d554b25c6e30a65bcd0193d721de9fd31582830', '2b7f03500dfcb101806a838fc817782c4967be240cefc6981ad909ece130235f', 'e7c98ed4db5ebccbc0efe9a2640fcd23d91d725f0b5b794adf10cb9e79f43afb', '68368258b9ab41449f5a2cb6488dcb07d8af5909a602afacdf65022375caf440', '4cfc45f5254befba2ecbac2d63eb7fedea7aa09ea01a8303a88addd489eedcdc', '9f20a8b25c20c15199c66cea40a11510ed13e949bf108e0baa6aa876895fa0a6', '1697875204f0a96a2df2f012b70e6b9334cff1b1b2a62d132f86093b094e7c3a', 'a6ef8aadc6b59e7ccffbedc36955d4063801a4db4a79b07ec6217a2110d64aac', 'a85adf45200d26d003b25fd0b7839313623f5121230e4ee5d2e7c0836deaed83', '7e063dcbf5860ce169340d17bcc51e0c84ff8851751e72763846a870490d42cd', 'f2705bab06d62a6a751d7df59d3530254895c33f6a9b50892fd16e2009eb5d18', 'b0b815a7541cc0417c39bb9bbbe2eccb4d04ce294f2968

['060ab242dae0dee598d0c971c03484e86e15fe7657be1b60331fe800582401da', 'e945c8cf9f25fab64da97f7959df2a878e1c1d03f6bcf5f83722bf7047c80251', '5a426be5b00ec430a9132b15966afea85cd466da9fb0010f8190c2e79a88f0c7', '37d93e3d7c5e073fef288c10a9e2ee2d4f36c329229b3e843819c9bf3e3d76de', '88755a6ac92e6e6a2081cfacd8d0fb9bd16f7c7b59be837c9514d6b3183f02ad', '705d015bcfe536f553ac4ac73eccbdc737b32454c90f50b32142faaca7c6e27d', '087f8ed8fc9e94a6cb5bd6071f562759a9e1e11f249a7dda78876b8a3fa45922', '81ed1e7a9a6ca986070ac9a7588b5aed55f2440aa56a39acee587eeea29d32ae', '791def23fa3399b4c8a7044caa953af584b63ed4e9fd33459d28de98b7508392', '90c610268fa80581af2ca73270ca68107b58748b0ff2d45f73ec5b3ea85a126e', '9b829e7a817a8664841f88ba953f6aef2b88bca78fb7fe9b60c616caea0453ba', '2c5b6453c435cd784d6b92cd6ae1a5ac1d691c3e71124ecca2252d95abf3bd9a', 'abb6a50d574d6b7028a9aff0e717121127c6fa995d9a1f88658a6cf50cea357e', 'beb4f400a90e7a3751b3c49bc7ab05858437424772c933028a80e97bd4c95802', '6b1e1be6e4f1c13684eb6c3c06dd0818e66590c4d98e0f

['a17b4af5decb27b5cac704cf7213ba4da6d539c15bfa61194b310d80fc98eef4', 'fa7251c37c5aaed4abd8c08aad88c14d7a4c4376154e29695ceffde9ca3bd686', 'f95cc84679679736e07241a641b69e0ea4ae563095a8d8ff323ef93d93be34d1', 'f74653c5743b14be6ed3c1d79570b42fee7484d66604a70b477573379ddf1399', 'e22abceb5142e15fcb198f1a7770650fe612a4aac8a79385c49af64d8dd34da9', '0962658e07006dc0b6d1baec3038b74d6dee4523bc8e0df51e63f622cd155fdd', 'ad564670f430884946897e59f91f8a5eca2645b3707f07b44fe1f68ec18cccc8', 'ccc098448e9a3f6b219e1fdd5602d27b88818e4f4b941b92c84e62f741f6ceab', '4a6933d80121865597d5439f31243611d7c6519932ceda9a23716cbb52cf30d0', '128b3650bfdc966e959d1987e11af00d235cdf46ad38b6e27b6c0095162d151e', '173d6391abf05665eae6011311b38b968e50d7d2b769de42609b199b06b4a2b3', '6dac65af231b54c0c6f54b448f3c216115b867ff8b1c101b72a9a30e43f585c9', '93ce782b7d6d0f6120d39d2cb560b3d2a61c9785445827efa224a99fcac2644c', '8ffa3ad8fe8ee82533caf4b22fbe51263ebdbfdec5e566306798115dfaa96e2f', 'facda1af501b909a64e33e8740edf8479bf6c49d9cc832

In [10]:
dict_id_anon = {
    str("2015-10"): [
        '848d371053f5c2eed1c84b443e4e3a6b9bb50ee3a949cd80ba50b8495d6f3d6d', 'c83ec907476a3101632c878366027ed03068d75db8accbb50675a818bae944a6', '0f27aa12029fd459c6fe3195eb801ca37acda08753ebdc9ca5a2ab7c1e791706', '3fcd9e6d2295cb93c852b9af95f8fb0fb7dc242b9557a0df45af33ecf9107c56', 'c60a6e26f74c9be0cd8b3f5fcdf8e20ae2e4c2efd20f3ec78b6b9b782665f201', '64073b3845fed8476608b0ab5a9ab52c5869a6cdbaad5c5f0faa93467d0ef6ba', 'a5e1589dc872f333508a84f551ec37aa3b701080fb47dc3ce1eb9c7442bd0599', '386d7405944c35803554176ec361cdd789920d1b1d538b759438f96acc9f58f6', '7741a739b8d6bedb5ad20c247f16f4b3842f75b5e755130f6e134e5edac9036c', '779150f54f4d27efec1b1a0fe1a84ad8f6502b8f8e1fe145d4fc8c614dc9364b', 'ec39639428c541c343da55fd9763fac38251e53316bc2926e92dd16ba85d196e', '79d1f4d0b438aa92efa4d60b0f12bac6d65d3632f4da59b25381376391726933', '4f7c38079e19d8622303e7891d4d4e34a5a3ecefa556676af4b28f1ea4ed4284', 'e77832d6d6260526c9047e410736ec0d93c9a7ae38184b08e13070d135d6b938', 'd183211bee220bb2430d38199362e7ff096afd1fdb87c2c7d28e03519e92ecb8', 'b904edd85f68565c730dee5edb2e308163e0bac592c727ed48e8f91b2635886d', '73c77e8157d2a54235aa39bf9cadce8f8c0baecf80ffa7243802ce7bc8ce7db3', 'ad1efdf036bd557e0f599233f39a5233bff0f939dc141b55c63e5272a73d0ea4', 'd5e133ac85bd60ed15e8efd0167dcdb83a26fa4d4484b570f0481d6e22198210', '6d48637ca980ad80f5f3481b3200c5e6108beb6f3339ecba8dfea81704f8970d', '3ee8a6ed552a3f90d2770a9e343ef2c3bc787eae11dc6cadf008412b027c53cf', '4a9e4f5361da222e761d7f654224b4357baad50f74a0c576eae4aff9ae525dd1', 'bbb4b8d59ecdda3c5c1a9c89b00cedac8ba919ad9f14d85ebd31cb67fd569266', 'a428c7332c4e1d4058f4ff79819b96bc7d87b7edfbce2715ec66cd8c925d9791', 'f39f94fcdf0f3deeb076bc51f4449f49b64dd75b57e63be7f9932dd55078c747', 'b90dfcf865d0c6f005dd9eb32cb4d37893857aefc45d655fee9144f1ec726434', 'd0fd4a933b11af694e336d5fbbdc3aa22fe237f8dac685ea0061f2a99b777517', '6b02f55b0c15018f5dc4abacfe45f47c7ae71356d8d8b13a47e43fc58eb1f633', '7c531f3efac8475332b4b8d7d0b254703e079acd24db58daf5539fa2064cd7be', '36a6263e7f370ff9f19ca47fef41f5af645f26b5e288b48d8362e844c6b46d64', '8845c6e6b27906dca314768be33da1873c47571b69ba2fbd6505ebee65026469', '68fb3368b5dd9d679f7eb6d7407392a3751e36057619176ba8f1fabd0bb2b91e', '551827512b1ab0517a4f9c65a864e79001237cac28b30981c3666c2aa659c3e5', '46c4fedaca2363f4a2e856ffbd67d862cc06f10cb726e5adc142062829de83d6', 'bf92453c24d19e41f02eae847b7694fa5de49dea8e2df478f413b3aae34206a2', '8a6d77193568af726427cd545446de0af90dd2160eaf7551718c0efe91766398', 'fd14b71434f6ad9880e3913c3642aeeb7253292943aab52c85cf47a97b6070d6', 'a3f2b5233a957c5166b302c3002d3947adedaf5142e23813d69533f8cc27b7e7', '40f2eb7c93893f48c6cc3fab9127c85332c8337efe03c86c1990bb8562b231d0', 'd36c097f54656ea8794872400c8874ca090b178aaafd42149b84b1580fe3e429', 'fbd7255dd3701dc51f6f3760e6a823164e677f026fe869cd8ee8c0c645e9ee7a', 'b94bb57f342ea07f338bf497d13d2f3e7e9703c7fcba133a869077e7f5702dcd', 'e1a7236b2325a82410ea7293ab9ef8e2d34abff1b0016b0242b67d8f093d8389', '17bbffa5ff9b37a7372477aaa03a617fb38c8771b4375a79b4eeb21507c04e02', 'b7e9540df1edb2c7fc748773b01326a4cc786618189abdb25093f6c9fac84294', '78fba9bb9c9d80670c6ee55061511c3b08c222af2be152f420c7b6df37b41dc3', '118b49abb428a147227dd1db23c42bbf9d8c18082bef63a1f1c97680ea0c0790', '8f8a1fce17c5ca9bef219d17ae86f16847befa090bc9dac4adce308227a7f725', 'ca78c4a54813dac580b527839512af25c543b296c4dd264686dc6f0cd90a41bb', '69c962bb49997ae5e10cdd95986179a8d1ec9c8d4656fab20e312db1ff812acd', '58ed186f73033470e713baa4cd50d59cae44e89bb4c29078fab2d5c4287935f1', 'c8b4c2612cc3a968adbf183ad2ca54dacd2a3b50bd84e01f6d639eb9a08cf797', '8940f203442bd247008bbd740e39d4fce292a6fbffd78821bab19c1347e20530', '69a24dff1c9229dbec252e23a447b918a077fb85c310e39500594926f40f9945', '83f563c8caa60e326fe1c72af9c66be1d7d8bea3576ee22c819ebe1d908c20a4', 'baaef14fbbafe27df523144b34261aa4ca57e5c1c416d3c6c97653fd526078ee', '7b45a8dbff1794dabfb838ec564969196176221fb2be83ec75fd4488005c0d70', 'a6938534ae28b3c6f6c4538725740976ef2c948e7c82915324f91a934f0ca5f1', 'bba88f6a26823b046720a0647d5559caae31510cf45e7a85eb9c27dbdb71661d', '5d9a4762320d0178201625138b94b63985349e20c58a8f2808602d35198b9178', '8b06b9a5c8c560b4be6a00063b176f01a9ae431f6e8f16b3dd3f6dde662b9fdf', '06ae95d85421c470984aceb6fb9510012f748d635644b46a8c5c90f76cafbacf', '82ca1bcaaa3756f6afdd67331058609a223974d4048e4596216114ce38e8e9e8', '5662854c0a4dc8aa47999a708d62defce2009460c4329177d7c79c0315efab8d'
    ],
    str("2015-11"): [
        '4126ff0fa81ed92972a64ffbd70ef391cb32568862bb52b2b7d08032def9b3ed', '774dd7515528cf048a4043c319f97c16f2441628c7feacde3d980e3bc2b3d268', '5f0db8f2041ac62d135a144a14df4e32ee53504216c6e3f6918ecd367c0ce407', '998902fcb1a27f20252fdc3f4205d9f3083df95719f9d6e2cca7ab17ee99b9ac', 'c6c4bbce0eab401de036ea343580c12e4b0fc063f710d84d10ba5ca4ae913cdd', 'd5aa2d041f0fba228b1e45aca6e14acc504e6703efd4942d736b8f248e606259', 'b6769990bdd3ed2899308e0e9d32c69038225d52b0c43b8c655217afed4868fe', '98e1352a702795bcee4f43d4b3e1458114a89f6365e5ab9e9dcb3af1f9b6f152', 'f8f173c1d5186378f9fba62ab91cde98029ee962fa44a917fe5745dbeb6938bd', 'c9839d3f6981100842d92a3175f9afcaf9e39d278e393af3137c0b1742394dc1', '38d784d7a67c1241411eb607ba68ef03e0034c3d277327103b2fbddb3dd0cea8', '6b65cfaf03b82a9d52d95ba8fc883c99ec93bd852163de4af8a7ab551fa0e81f', 'b557009b855a94db62c968049418bb9f4e95272bb6e6b83a41957047dfbaa16e', '70c1c57fb103a14a1a22aa7eaef705ec70b13e7fe60419b9e343c7cd8981cbf1', 'd1ba57de4908587e5c5b9edfea1c58ef7430375c9ee54d08159fd12eeef251e1', 'f6d44c06f1b7226111c88e5c98b9c3e779cb7946bb241675780cef12e6b48dc6', '3f9e1dd15ecf6b9d021fbdd97bd6570b52cb670e53c8693decbc918859624895', 'ccdc81673b096145c7ab77c0824ce3d7744db9c6bc505c05a94449b9bedc622e', 'd3ae914800908f10cfccf8739b46c47c37de95b904ec3527569556e8dbe6e798', 'fa19ccdf6418d476a98f4b30f55820c688ad8619de362d65a2f571ddf82f48ac', 'e4e033dcb783d882f8d7e3849be6b86685c550887fad11ee7fb05d63c2184b4e', '0c3dce15b123aa94e31a9dd360f16a81dc2ab1394bf40c5cd245c46e3e922851', '846cbd59974a024427e896833ff7c1b5b993966b07b7cf18dbc44adb2917e83e', 'd06bf6246065b85e1a7658aed9298cef4ae45bef9a2f353a480b9db51cfd6ec9', '3e2df642e93d8c73c85d291eb408e212d65ef312793ca22cb16480fa6bec580f', '040c4530dff20d1adc5e779e30b0aec22b2fc008ad7bce57d1d8d09f2f4d2b64', 'bca8ae29fb5db19c4b395aa339771fa0db364e287944c2d017a4c595e9b4d426', 'c1944d043ee4ab145a20c799d65caa78765eb50f34aa74b7eda8db92b220eefd', '0cded97351a1babc31db6751636d36f487795c80aa3ad37bb427475a3c21f852', 'bce73ae3da134e49e166971b2f9aef8b4df84c30b52f2bbbf7de694ce1ecd1ff', 'fa78af1ceb1275a35392c3f9eaf9ede758e159aeb39325e80e350c07314f950a', '3076b5cc19cdbdf6f0599937cf46b2d5d73db8ca1e9960192bbc5a0d493c0623', '7d5714a25b336b7962ab95f20e7fa4871285dc28949892b5beb54dffee9d1569', '6ab69fafb70316fde7bbaa46550617e02da8edd47af5f1beb1aa9950ecaeb75f', '3bcfa755d611aa889a25c58ace5da210a7f2ae05a486e41f8a1c44bcfea1291b', '6ced33d2a27902ea3f2985aed3cfc31feb6d548aa5830e6106107bf5f7f95743', 'd18eff58361171c113cce57a4049aaa801ae4f17c9909496ded7536757b2e388', 'f7515239e31e4a5a8927baad6a6d966f8ca3bad5a7f79a306156fc9a7a523c28', '678e3c7d19c6fe2919eb9dbec9252de03b4b52c730fdf9615a710dfa70fcbed6', '536f12e470b8bbf99a0c4750babb24c3eaf2ec1bd94162e5665999585f80ded2', '431956b335870b62f732f7096f35721a4118f1cb736b8ddfacaa5a3b4a2e640b', '73d50cd10b468d8b3ac3cad57a2307959baae7ecebe9a57c002b615574c5ba3c', 'e1e3dc0edfb0c2ae237c0fe0df2839cdea160b7bf6aaadd1b2a470ea978f161a', '10ecda9dfa013df6f8dc70079a352520f42e5ea9545e887b6c3aca1a4261e9bf', 'c1ddacf3f5b744b9a119da643f05a36ecfee51e46788a14b98a580dc307e389f', '321ca458b453bcf6c0005df09ade7430b7b41111e4d903063463e7309c364efa', '3e17369b538d0bc934bd0a92ef05eab05854bfb0d4cf19ffd42fb14bc7bca730', '2d2694a502e7ede6bf4d3924bb324f42c35b1e0b881128d6bc6037bab6a33336', '47a12a153c051aed2b7e8250a1af4446b91cb1cb73fda40d149474a10ce70586', '36a9c4da7aee9ddb283e8f3bfaee761f40aef3376d976fc814a94dfaa81e2bea', '520be3bd213da3dd51f007a6ed7e480894a06f041e9ca3aae27a243d99df12e8', '9412cc7026a11ddc5da1e1e5a7a5cea97d8f99144d64b7bf126465d823ddccfc', 'a14dc2186cc47a6e5ba3b2657ce01fe01647debbde0ee23bb531a837349153eb', 'c61fc67ff2b1b90bd9bfadf9edff0f418bdd36bbb12d4e23e4393a2106481a00', '011b4770a1952125318a0a9ba440769f984c58e5fffde42ddc69cff8b0d757d3', '960e1e875a7c4e8d0625416868428974bbd101a50608a7f99ac90614c362fc8d', 'b281ee0ab459eccb575f9428f93d0e21cf154daad96edfd07ac7c736e71f8001', '83e0519a615caf298f3a98e3a55dcc48a6f6aa17397e84f55bbf6ee221c932b0', '49cb64ef8b17964d53c65bdf7438a1c4db7b5568783ba301b45f4fab37d5cfba'
    ], 
    str("2015-12"): [
        '3cd165eee7b041da6f88ed3685dd1f3793b3c5cb87e523310b58711983f3f754', 'e8b7711943eb868f29cc108e55627666e3d704b1802090f271d5d0dbe6689a84', 'f0c5ee05ff0751db09b41dfc1ccabc3dfab86d2c579bee76473938da511a12fe', '09ff6fde378760ad033445232d554b25c6e30a65bcd0193d721de9fd31582830', '2b7f03500dfcb101806a838fc817782c4967be240cefc6981ad909ece130235f', 'e7c98ed4db5ebccbc0efe9a2640fcd23d91d725f0b5b794adf10cb9e79f43afb', '68368258b9ab41449f5a2cb6488dcb07d8af5909a602afacdf65022375caf440', '4cfc45f5254befba2ecbac2d63eb7fedea7aa09ea01a8303a88addd489eedcdc', '9f20a8b25c20c15199c66cea40a11510ed13e949bf108e0baa6aa876895fa0a6', '1697875204f0a96a2df2f012b70e6b9334cff1b1b2a62d132f86093b094e7c3a', 'a6ef8aadc6b59e7ccffbedc36955d4063801a4db4a79b07ec6217a2110d64aac', 'a85adf45200d26d003b25fd0b7839313623f5121230e4ee5d2e7c0836deaed83', '7e063dcbf5860ce169340d17bcc51e0c84ff8851751e72763846a870490d42cd', 'f2705bab06d62a6a751d7df59d3530254895c33f6a9b50892fd16e2009eb5d18', 'b0b815a7541cc0417c39bb9bbbe2eccb4d04ce294f2968ffb9a2da763144adce', '4fef4ed5016170cfb93ecec3d6cdca351267a81f97ac436a698a3fdeeb8e44cc', '17a39cec18ca923b422e25d7b1845bd4a454af9e950c904f6668259a51845e05', 'c48fdaff0249326ff06af9f50d02599547cfd0496ed9785599d8a64abc341bb4', 'f3621c61d7a720fd15275e9f20ed9dd8d5c323addc84ee5b3d830a2f0705da86', '4bd27d1cdae44725f31bf58983ba7de24cd840a1dbbdc24277c2b0cd6362007c', '2d674616de64969ebfa1db8d8b56707531ebcc501b7184a290e21c5fc10aa566', 'd253ba771084a510c650365a785ccc4c3466479aef1e73bc9298057a113fc24a', '05b5121c30d6c71bfe2ca627231de99100c62f121a837a480b89911ff6627572', '5db5e4729f04f51904442357d086d131aa72654e7e3bb1650e8eb24f918c31b0', 'f483c942840253e691a36794b0cfb6ab7b90cd200f35c9d7ff9eaa0cc2f82269', '705e21bd29a5562bd6184e79e0d27069874376d896fdf6c5e65763a6f42aa3a7', '52fe4ce768e0536a9065a8743670f807f1570b7bd9399ab3c4e746b31d91444d', '8ed3f03073497ae367d1291dae168ac5f559acd00009146f910233df60035f72', 'fbeb1262af1cfa8053ecddd2c8dba7ed0557a32f35c6713f2badde1997f8bcbc', '4eb15b1918365f69bb51a5634240285cbf04f6490466a50e7c00608f96600a50', 'ad337a83cb07ce4c230656b203d18209a3e646cd89d935572706e8e8d32cf77b', 'a2d6d7e4dfdf585e82b08843bee1ce5ee3d4646a4242453c124f747f5d57fb9c', '14894ea9d21d4a775de632e69f998fccf9e9b83ac2c06408b6570daf444cb1e0', '1a1e4a5f9affc74565108f65aed11eb4a3d3262ab9056bccec2a81fa4caf6d1f', '94d744876113995f613d2293b32e891a5dff2020fbac29712658e2ac73b361d6', '48c42e60bc7b869881d4d3b8e0d0e63374e41fa3fafafdb58d8acde65bec0f9e', '63fdd26dca6d1bce98728366c60265e645ec42242b3716a9fb57f30feb18d517', '414e9994db0a76b8b6360ea1d99fbf237dfb2baf38000ab342f0ab31687b0d5e', 'a12845d17cae5f94f3e20ed0204dc799ed6b2f4f3e59ced7de36c03e6c17e8d8', '5570089492407dbad42d72c8ae1a329eb86265722776b326ac3fb4e60d433ae2', '7ea8eae154b825532675cb78525da8ea36a091e5ef8350c7acbdea8569da9245', '5f1678a77016e05b8722702497f83e334c8e487afa6f12f79eb71bd1a354b4df', 'e9afa805e9356cccb261a9b3fcc760b395b3b8b6bb8330a4533de7b9a49f1ca8', '77a9cf4e79c190d1c3a72ba6ec2e38e1210ec3e9f2b1347578f0d36c9f1b1322', '550f259f7847e1cc26bf719e334cc8dd6384bcb14725e96cd829d887806f362a', 'ae8eba5bf2906e0aa2cfc772c8c9b02309447818fdefa0e9a7958cdcbdb0edc8', 'a268ee6924913585f25697213834215ea88683613c8aa13a951dab15755ec19b', '6bf53d1d7955157ebe506906134f79e4755863b47ccf9f490c9e05ef8136e8a9', '4357c92b01f357b57c1c29c0be696cd60ca64a9c010290724130c416819b4700', 'f4842a5b852ddb09c08b334a3f3d91f592e0046d8a3900b09c98d8de4638a0ff', '30ebead398ce72160d9b4b2f11952fc8c0c61dd8637c5f50391431f16a543b26'
    ], 
    str("2015-13"): [
        'f4a94b1dccd2a8bc36cd68c7bcc30fc8b52630f91bab37e87694a82b304f3faa', 'a0dc29d644b843f788ad2c49cb6a160bf1fdb0983ac67b312122651161ee9688', 'c6b976fc657498bfe96fc05d48d8b7ce6e671b92b1444ee47fa67ed378a1c066', 'd10fc19eb2f03c4f569550029558a6d05b661e34f8aea84a74e7016583df596d', 'bc5ece5852abde0e2745c51d84163ab0752ca146d723518a93cd6de1c479b6de', 'eff85f2268e8132c8e3fb4e9da12f97b6a59bd2ad34467f43d13518601fdc47a', '3b6c6044ade3c4c3061fba3208b07f929c1d281a027de68f3a2e8d69fa21d1a4', '1eb365c3bbf8688572fb6c58923ba7559539700fec55d0cc2bf44b076230442a', '725821fe8cd7eb8038f1b16b668bb93bc1b92d3a383a4764bc2bebfa2a91b4bc', 'd5ef28d65fbaf7f7b934b48305ef03a7ca033f1da7c171d8c3b52a696d6d22c4', 'ae5467e87120a28c672b45d7c90546f85e0c4e6dfafd7970716f6f9967dff229', '3c1541a282a08d6524fb3133cab4aa9a5700a1b48e99e01d0174c56a74295a73', '11dfaeae41371d9291fd18bfe7788f061658be5a96c8c89f4c8de5066b34dc61', 'e8c640b74fb89e1137529db1acf262b0d26fea52d1d80a0d34f0ca6f345396a0', 'f2fa08b4eb2f4dbe9b7477bd2f1b798c6741bbe1da219b62eedc027c7111753c', '95d9e59cf5542ba2307c35110c7333d437657b69354a9bb0dace628d82dda1ef', '8b56f1449c86cc87c1f70f2ee1842eaf9367e91b731842d88fc7aa2a7eb393cb', 'f2f77ac0ac07b942389bebbab8b5ce65ed988d312ecd4fba21a0902dca9fc7ad', '666a77b0aa835614956fb8c61041570eb5c4eba7a8847d4731aa0c7cef4790e0', '31c9f583458d0018ec89eb2700cc61866c0acaccfa50735c462b8efe7881faa5', '60ed7f666531dcfc37fabdf780cd519a51bcfe5f51ae835b228aac35caa4dcf5', 'ea4cede015f0160d68124a3b9a76a3cef254b2895f0963f0c03237b411ecf940', '39f34bced9a88c4391a2734306fe5dd3c36b053ce199cb77d5c3c2154351febe', 'eb97ce4c667ca4b83ef02ac79b73551ed4b75815a6b65c80b4656c9883d6655e', 'a815eca256e09ef933bd1277a27cab36db00943c113bb27568aa108cf04081b9', '94a745d52ffdc41cc308d8a8205016cc9ec84206a4dae269a1cd83c2bf40493b', '50452d5305393148a63a1ed3c2b52760467048b45494f46abb8b737b0625891d', '7f61bcb02510cb5e3950e58229357d2a294fe62735bb000c522683e0c33c7479', '2627e2410e01ce41dc8ded1279816baa184dda0636250d7fd508359ace92262f', '438cb897ab153786dd5554b26aeea39adac10c6f8ed19522d33bcb851a2276d3', 'a7d356cf9e4ad997e129da6c711f01ef8bd52906ebbd8e3d8d658c560525add8', '87af371172c633a3f074dd1fdc24cb5e5165bc2f8e591a0a71ae22e96822a471', 'cbd27451979ad17a0ab44238ac86571d17dd7c07b73f1bb20011d42f4a5ddaaf', '2ae491f2f42244bd5d88f8f94dfb4ed600fdf2e0905941620c0ef7e02c691c82', '27e30d7a27d75ec346c532f88a8d6c8e88977c94fdf882a6e57e9c5eecb4285a', '7017f2f116b0e4f299ea7b8a5ea88e878c5f74d7fe8eb1fc7d2df2f4f482ab75', '407c5a10b817be295f6e228c3edfb165edaf1c53a1450b528dfa801c3effeb89', '1fd6e233857629ee08f63d6e5e6a49a898697837f2edb610e7d478575a4c6bcf', 'f069a7f7617566bd92ab56728f8f4323a05e0bfacfd41774a3094e21c609d99b', 'd5890d6ff03a3a5f68f3ddf438d67cbc915ac63965ec90e56bffa37eb619bd09', 'b1ac79cb7411fa91ad3d13ee86207a515ab10a08179293adcf245ce1b693b1e0', '902752e3e2ba48a0d18b4087d03dfbb2e267eca2d0a14accb2398a31c2037760', 'ad0daa9b184cc5f92a3ee6ea9b6224c8bae6cb89f0005ad6852386f306581a27', '685ea4bdbeb93c4313bd6b357eed479f20dbe5d9d22d89adf8fbd2a9949bf6e9', '756ee83028c7c564e14769ce2a6b905978c61e3506997e4d7380c6d24546ba33', 'adb155c7344405193c1db6ae07b89b6ebc0f05704c53ccf0afe1d67914989271', '0cae0d40bd5874d9b291ed416eaab77b2120224a7665a9e2514abdab0c78567d', '629c492eb138d1466cb5e6a2e44a1c6f59110ba6ab84aacbcca58304ad248833'
    ], 
    str("2015-14"): [
        '432de568f4ff0e4ab0a9f340a5bc8beabc5007fd19895acd146af9f1ab80774f', '194069ebdd682d30b42c0e4a936dd8c98c732de17b9d81cdd896e87326d4d5ca', '812c6bc9602b365eefc76d1c6af5ca528cbafdad682d800c09e3ea4bd23aae30', 'd30a7159872a37a17f4120d97da91c2a928ce04cd24b32fcf6fffedd7ce2db64', 'ce6b071322aa4c1293dfe12921dd87d8641ef0c50b0d8ade2512f9241dda7b5c', 'b54e51111c7e6a611f1e9e462829d67f686216073e9002f6676327fdcd363e44', '3ba15926ea149278a9915ff7d8eb52da37121a057695fadc05ef2eb3b4cdafba', 'e78b811222950cb94981d390f0d0ab8438386fbf6480bcf53e21308edefd141b', '454e4364d970a7413a6f6dac78b0aa93b2f6096858107d9d94bd90241c8a00d7', '719817007c98de377572450b53a148dd27ef64a9d1c6f8afbaa165f30cef6294', '1ea950e4aca3588d2c69726e0533b0bd9b9e41aa9e645fe3a5069e6da297f5b6', 'b83d76e54f1512eb03bc62adc0594a4437518ac33e94754632b56d0489c54ebd', 'c8d79618ca007529518c9a1bc6762b65240940e1970d9224fd1d1381b0a07993', '8e5d997123706e7d10962bdfc3df07b3c2400c1da8a5fd42871609311788d691', '7d6ffdbdb5bd873145a5987267468ace11931daa4b6070283afa034391bc0ac0', '83934a10619669ccfe9e6e501d0061d606526181737f1b4b0935f1270f867f1d', '200035221db7ae5c77f9cada814c71d0a551987b8c6ee072fb40b8ff2165c692', 'dcbbff2780ce8d20205e672d4b4936c0d776e1413c0831a7cc0356809c6aa49b', '5d5b079a40e2a8e67c509cac4f794215b7e4df4f03e7c69d4a4fb0eb15603406', '8b9b0c76cd788c2802cf31bb4c74e48575462276e7c19387f22f2f9c7b202838', '661ed623415441643e0a4ea86f284037b47c9fe770fc1e3a65c5e90cc00b37ba', '1780e760ffe836c70da9746802c7000ffc225f7b2ea0840fd2a30f6ed6747ce3', 'ff72089406c3bce99095352a87bafc7f545bb9657277e494ba954439e3c54554', 'dc6fc7f5bc236dd956b0e6b36261af5c1c78101be3ea7ee6bb1bda74b8b3a3d5', '7158a443927d2da9a3646cbd947324cece7d757103ffced510d3af947f243782', '2748ba09199f56a6b44ee9fc6fa2ee03d7c843c3a0eb824add7566d25848e46b', 'da10ed2f69589e22bb193ecf4121999f2330d513e82f1605760774e9f915f6f5', 'a541db7988aeb6ba58e7e0a4d5516177ff8119b00a7e1b056865c2f721f78488', 'b832eadae9b2c25476d70df787b058036b60dccc0044e48a2d6b50fe559d45e4', '9ab2a8bc20c8862e0ed513d88505e5d6fa03ccf91c4bf804a9ab71d679578816', 'a2d2bdb2d23e9996f1295bf921892c703da56100758bd14945c5d6b309070f27', '28418361ae29bb629414857784c08cd19e3ed84cc8cf3d0b90efe5e97ec52a1a', '3be04c9cdb4f027ed7d4c5bc6b323aa6610c7deb205cc92f8a4d96992fdf1138', 'f91f3928c6f7594a472104173a2fecaa7aea41a8f179a43e515490bb3483edc4', '52165c9cffe6f374c7054e0d92e282d72d4df1d3c6476bd91b3e1dac3389295a', 'aef8d588e19d9af2fd88c06d62fc3370e5f8ee0c835fc695c25acf143d1ef555', '2c2610a0e83be48b97fceb9e40cbfc22d25c68ce20d5e0cb51bcc0e750acffe3', '9ec032c753f68a34c3e25d7ccc7379fb40bcee500e255aa73a506b67cead6cbb', 'f2f38569585686410546a4ce90bd4a9462aafc76a696d62108ce74f58c41f92a', '8c890936afe9be72ecab7799c8d59f1cddac1e3b737d3b2a8d83cd72fdd85e32', 'c62a640eb9082ac9d3bdf2db750adf987450865d8d78b1d15152d42487b4f20c', 'e3cf291de93678fe7a1473325bf3672ebe820dd4634da57b58a13912f1b77aec'
    ], 
    str("2015-15"): [
        '060ab242dae0dee598d0c971c03484e86e15fe7657be1b60331fe800582401da', 'e945c8cf9f25fab64da97f7959df2a878e1c1d03f6bcf5f83722bf7047c80251', '5a426be5b00ec430a9132b15966afea85cd466da9fb0010f8190c2e79a88f0c7', '37d93e3d7c5e073fef288c10a9e2ee2d4f36c329229b3e843819c9bf3e3d76de', '88755a6ac92e6e6a2081cfacd8d0fb9bd16f7c7b59be837c9514d6b3183f02ad', '705d015bcfe536f553ac4ac73eccbdc737b32454c90f50b32142faaca7c6e27d', '087f8ed8fc9e94a6cb5bd6071f562759a9e1e11f249a7dda78876b8a3fa45922', '81ed1e7a9a6ca986070ac9a7588b5aed55f2440aa56a39acee587eeea29d32ae', '791def23fa3399b4c8a7044caa953af584b63ed4e9fd33459d28de98b7508392', '90c610268fa80581af2ca73270ca68107b58748b0ff2d45f73ec5b3ea85a126e', '9b829e7a817a8664841f88ba953f6aef2b88bca78fb7fe9b60c616caea0453ba', '2c5b6453c435cd784d6b92cd6ae1a5ac1d691c3e71124ecca2252d95abf3bd9a', 'abb6a50d574d6b7028a9aff0e717121127c6fa995d9a1f88658a6cf50cea357e', 'beb4f400a90e7a3751b3c49bc7ab05858437424772c933028a80e97bd4c95802', '6b1e1be6e4f1c13684eb6c3c06dd0818e66590c4d98e0f7a00b99372d320a353', '7897f0786927eaffa09c1fc1151081ff064f8a14acd098536a174437c378a914', '8b1abf89d746515a6e2962063d82ab0583543549fcab6f4a9ad6a23bf47b2bba', '98a579f0cc7b0405e425b749924dda8d581546f1916477503144c6f5a299e4a2', 'accb1ad9cc10c228370f2391ea3a33bd6e4940a1ff74fbe4b8a8ba63ce0fcab2', '404ac0318f1cfe85fbf8732aef2d530ffa00d6d28ddb8d3408cb48b4c5e0e2a7', '840fde21f2573de41995bbd8c07b3bb43b6ad5d2250a1d158b116a6628c476ab', '5ba9f9f74a71f513e310b7b2410e92166dc8e19f4ae13479be09dbe01361b7bf', 'ce83c192f1c06768003aa936b718d3e7498425fb866bab73349036df6dd6febe', 'd3db4a152ae9e5afbbd32e24c6a3ed65a02852ab2cc085e561e422fd07eb6371', '381e3fb85b82cc72fe8bb233e6281b7df8245aee1bf4c2c01e98119436cd40d7', 'dda08a22bc557e96c5ed2159f4b770f9adf265d037166987f0728ae48940d12b', 'e0a357db77fa2c9ceb520d9956d4899614e126e499bf09fe5a04e56596251305', 'dd4971b8466d0b5056b4771df0c67634ecec45cb3fa7a1c2c14de36f655b5f48', '46826604349d0144a183010c30aeffc87993865310a6c3d70041d6b6204a23a5', '363c156f94abf885b61b0f572eafcee0624482654cb7e41856012629117bf9bc', 'cf5048d2f7dce48a7e84e7472dfea269066be1ba94649fc5669769ed28f32422', 'b7956c00ff91598a21472b1f974bef588c55b0555b475bf7ee016796224851c5', '2ab8b3468935ea899afaeaaec8bd54704778a76e95316f2a0efd15bc6a085786', 'bc0f1c6430918e44d572d33c90170c4bab08f98226762a45ed4485b8f7472153', 'd20b8de301ed5a73fcc334479e30183fbe8f990abade060f640e51e9dc6ca576', '0aeb91432efa6788c31af77f4e5ad5fa350eeea116819325525ce42385cedc72', 'a4d65da444ea089bf5f7dd35d92d4fb18699acc0543669b6ce91a060d573ede5'
    ], 
    str("2015-16"): [
        '02008b1f4bf0b4c85bb315033b8b4b6becdd163f967ef8242bcd4f178708e825', '8c948e288bac6eeee8a29e0d3e936b290412b457b6c5f07f44fd546cb1419bf4', '9d6e834e4b22addc5bb2c2f7e881bc76450b2615846f70c1d208d57e12ef2680', 'bf52233d636515be383c00cede7beb4af5de0368149b8a201e7fafd1c75a1862', 'e9c9197862f8feb9c55bd981cb9fb7ff1f1c89dbb12b93bd858087bb04c8b7ce', '92de7c04da5e13ec3f0e705645aff8df30e48e0909aa5347db2c713a0cb74100', '10800712c8f0bf0908e6d30d650ea2b4ea2a17669ad4239ca69f292371dfe739', '0a1a5e389a4c09cdff525901ba3d4750fcb2a389d8cecd7be94f9e499df1f593', '6ca213771065af8a6af8875414e5a5bc161eb74164801faa4e54a57e6c84f0e1', '0a2c98fda9bc0fb73310bfc01f3205ccce2152bcfebef2e32f7c1aebf765f97c', '3c44c6d710733e95be72f2157b63042c835e5314c8bf1b021ba6f657de73ab5d', '0ee5b230783febb9afd8bd15e1596cfdb1c504fb6d5921ca27c795a111550489', '8913dc7264517a301b353dd3ba9f480be7a17e48b07527b7d46cc0286e3ff077', '9d2cbef705b1934e174e2fe78a760b1fa3527955e3a96c7917cd2df6e56718ea', 'b2905bf5752d8b080425c51097bac195778fe52d5031ea6dcfb793854dcf1c9a', 'e148620a33212e243addfde15f5a4bc58eda84d2314a67bc85f16b397cb29edb', 'f5b68b250b5d581392dd0f538d088d7ff413cca73619e06544a4725cf784f76a', '0ab94a0c21203da917b8e74c21c1e62f0078421044ad8903b289125a8604596a', 'a027b5fde906ac7b9ca1c3e5439c221640739e9cc1f26b0ea0cad30a46c75514', 'd59afc715c3daa01fcda17873436bc29527dc94d5d6e727abc70e9be7310f0f9', 'c4859e5d7b03d5a6560cf0b049bc249cce63540fbd2971cc6fc88919a01ea6e7', '3da07ced5dadefeea0e9227890349196e151865f67580234186d6031c933fe54', 'bc562c9669f48d98a8f179952f47e129d1919b9bcdb54bdcdc6a41f15fb70ba7', 'f418b0269f0ec3870ca7d0d378e1e4d38d9e17f37e3b7e329f73a237576e41cb', 'd8d0cd1cd15649d7de4178911312ff9a7cff74290e1545a1232d7da7050670b3', '8a71d4a52d333efb1d0afacce6ba62147b1a252290f6fc2d4b5869ee71b77317', '1a099277b139f9a8a8ff351e72510b210c850d9de1c047957ffba1138a8b01cf', '3d91fca3cc16dac6ee7c30e6ddc2b6189c1d972a10a9ec085a0310478a03e387', 'd72432de5f08db9e56176ab1f499a9a2ffc2d01a7a8556a76a5bb6af3c32aa5e', '2c99ff806998f29bcba014e6e1caeea59bd003e9a70557576264c03a04bcdd8e', '2c9707238c83d697853da3dd65398607403c17bd1db1c4e44154522ae2b95e1f', 'cdcc75ef9b3a80afbaff51f1ce53bdb3ed3b7665d835aa6e3ec0661296c317d4', '24f7b46b06a36fa1e258224d12e21e13bf67fab90f566d0ad75da055d4c61f9b', 'd3febbb353f1be0572e610a06e518c1ea0da0867d540b571f4d3a0b9a5b369fb', '916b4a88da754bc9a54a8582ce47327c4a22010cbb396e19893a1c58bf1907db', '731a45c31825349d7f2435cfc0e1b55e523edf076e6edd6cd6be19e705e0d1a5', 'cfe2b8a050458eb09d9c92586b3045a99602ef80c3436cab10dae31192f5e303', '447c03182fa690fc48d80f74b2e008e4b55ae50550620a1b6889a6d019b0dac6'
    ], 
    str("2015-17"): [
        '0cface865890473442cdc5c48f091904efec8f2096e2d22641d290d428a351d0', '12bc858499ae650d5c41a964d7de7107d65f7ff39474649a24fb1028697af134', '27edb10b4a0b4d805d190ce81fe76abb42a5c0241eb1c40ea1f8e832fa17f2f2', '42ad0df5d37e07530eb958b06cbc7c57c6778c087765eaaab7f22eb3792a63c3', 'e0f901e32d2eec5fb4ce459430ce004b4dde4f377fd526e9e86ab8ad22b38bd5', '6209425e47efad082770586755f38f9c7274008b11304a25391ff607bdf23627', 'f907f6dcdaf36511e44dd4c678a1f49d16bd5f99c291736b90f32bff9dc308d1', '53389478c1fce45571c6b97b2e06965985568fb23a043c3a33928ea44cfa8ac0', 'b35e3e6e83f57d3b10ad69d1728621e9e5957e73d403ffcf3b7b0a822f7de17a', 'ed53a72442cd2f5fdc72ed07c8e25d6ed0433cb34581557c461284abf4301660', '8906e3e79cc13b832570716ad3b59c2c7d619b5e5e1055494e9d5a1a57258f2e', '40f621f0a7139b8b7f8389eeb918f43a5a711bfd159133b1700bcadda91445a9', '18244a33734b8eb51ecade8b6f945c49611de40801e9dcc9c0da99be7614a325', 'a3a2a84725057cfd5db9517df051c92972ee3f37810fdd4f98e558103539c926', '2accac1f4daf2ca96d82cfad3a2732491229b8e9157e97444d4833ec5c709389', '188b4c32e37c8cb4fe2e86f688367cd43f785cf0148b11313e7b6717131f7875', 'af05404b4fe05d6c799f0cccab30e126af9ce9a4f88ab0c26fc0241a099a040c', '3d6afdd064baf0e341b873ca8d43e03ef697588a3dd97526db08655a885004fb', '6a5acd1ff55b0320c2890a71d688e9a5950dc8a102f8d713d6f3dd2a6063ec29', 'bf87d53818bf4ad5af33f976c5c9af982032d16810d558984a6fedf235a1b678', 'd258fe1caed3c003056337eee7d16c828a68c7e56df3f69a7827974d06cad590', 'f6e8f5e994dbb19e8c8a8fb329768812a748ae5ce60f7fb03687e647c3bd52cf', '3d9291219cd101e72077e629c6dc7ac632b8cd1b79a4a5847a7f1e1a90142d9d', '1255d3dcbb9eb4399d15cc348deaeffb663345cdd51c44eb9e7d157e01f40a54', '34673b86f504e91968867d707ba928e0b4e60bbc453d118239a8afbb4c1c2013', 'aad13cd5df2cb0c3c1eddba77f9658b5ee6ed665caf3c6471119c743c034d74e', 'ad0b20fba76038c21e8b9265002cdcedccc64d08eb31352ccf2a391a6687e03c', 'bbe6ff6a9eb7382794f3f7211b729724916aff6d5798e2664d6a1057c0fedd47', '0f827f1c95a0f1ccf4711b38c2c38f0bd62e2f2b49acd4559bc84c7a55a66b74', '224fd29d07164a9ded2dd0c7533baf17cb885e25b9cd60b0aaff14a384f2f7f4', '3ec59de1b6b79133ca20a1a7d37d928ec79fbc383572953b5944bab2acb59303'
    ], 
    str("2015-18"): [
        'c8d1e37128a94d14788c1fd55c63ee2521289de57eebd51e528d613e59551fd3', 'e5054c57285c03fda7830e4485ae8ad8f6e7d8d0b1e0e04e5012e16a1f54c429', 'c91b5d1da93196fea68cb65e9f03e146ab7648506624de33ed52a7a30d365b41', 'dcba5f472e970004ef09495f6bcd528518c2cc0735a80ab226cd85f9dfb12759', 'd03b032e078793bc5e8d0410620f407236d719b2e46e678069891840ffa4af1f', 'ac202f5f2d719771aca82742710338f2d82451f4fdd2492ca864ad354fbc356a', '747cb8aaf9fc73fc541a3c9a0f6b9bd03c82ecfb4b0c948cdb233d7cf72c5409', '70448bd4510c938d5cae3fbf4377f2c23f3acd3bc1232b92cc1ed318028acec7', 'ee77346b89fc8cba2682af604b0296916dc3578a010a6db3b59c99cd13617bb4', '6d5ebba3ffd2087a9405957a457bb8b69921e3ec92a746033c1c0d6eb5271300', 'c3ac65df1f5ca56c424ee324d0de452497fdd905eb864881be64d30b0f47edfc', '0bb08a6ea7a8ef7ea58f11426af79c89b5f18e6cee9ac3de99c3d9eafb701140', 'd1c4ba84672cee4ef8dd751e3f554fcfbe332990ed66d49dfdc1e150f8606d4a', '5fb365c235c219b00e32ce2bbd4a201b9d2a6dfd95354589a00784ee28c04fa8', '1688d7ea618d05a35969d44e343ca0166cad67df7fbb6e8f3f14df005274a9b9', '144290176f8daecb9de14ad14bfd0407be1ce871175e7b1946175d787577cfda', '8e42dfcb46b148d4207a656aace9756bfe418c81289eb5d1d4d99dff9c1fef7e', 'c692193095ac83399d8938125b03906997dfb12938d28f0aa3fd3eb2d911542f', '398c09231fcb5842399f317fd6647ee48c89953f03485dda162eec41e14ab33d', 'f6104b2c400e2f0416ecc88b0f298cd1efb5b33a4428dce28bd0f7d3e57cf193', '5606aa59f1fa88bc21c7562d275b698f32a7fa8e38e7395de3dc42001cfd615b', 'c917f5fa376c4d56a7c2de0405b41bde7d9bce297ecc3f1b241173a6fa44e152', '6878458f6e471272b6af9b23f0d280c1ea7494da0a059fba3ac022453ac60b44', 'a2e28840e15a60ef43611de851e67805027609aaa7ffd91e9eab12b199eeed68', 'cc69de8b681874f07678d2b0ab1a8512b899f610bcf5417dc919ab4ca475cfe6', '5a6109b0e0a03e36f3b7fa6b4607393cf0044025149f6097626f3745b005cbe8', '289b8475645ccc08b267c8ee1053172a3f2288bcf2908641430cd39f62a4ed42', '66b077812ec28ef07938025127fcce4b36feb43b4c27339ef1f566c715a84be7', 'b6c9fb93266b196ba93408688356eb8960567105d5c3b9904ffeb0046d34b9b7', 'dfc0fafff0c472b9916004b1e5da140a5875a938c7cfe0ccf425164db4487420'
    ], 
    str("2015-19"): [
        'a17b4af5decb27b5cac704cf7213ba4da6d539c15bfa61194b310d80fc98eef4', 'fa7251c37c5aaed4abd8c08aad88c14d7a4c4376154e29695ceffde9ca3bd686', 'f95cc84679679736e07241a641b69e0ea4ae563095a8d8ff323ef93d93be34d1', 'f74653c5743b14be6ed3c1d79570b42fee7484d66604a70b477573379ddf1399', 'e22abceb5142e15fcb198f1a7770650fe612a4aac8a79385c49af64d8dd34da9', '0962658e07006dc0b6d1baec3038b74d6dee4523bc8e0df51e63f622cd155fdd', 'ad564670f430884946897e59f91f8a5eca2645b3707f07b44fe1f68ec18cccc8', 'ccc098448e9a3f6b219e1fdd5602d27b88818e4f4b941b92c84e62f741f6ceab', '4a6933d80121865597d5439f31243611d7c6519932ceda9a23716cbb52cf30d0', '128b3650bfdc966e959d1987e11af00d235cdf46ad38b6e27b6c0095162d151e', '173d6391abf05665eae6011311b38b968e50d7d2b769de42609b199b06b4a2b3', '6dac65af231b54c0c6f54b448f3c216115b867ff8b1c101b72a9a30e43f585c9', '93ce782b7d6d0f6120d39d2cb560b3d2a61c9785445827efa224a99fcac2644c', '8ffa3ad8fe8ee82533caf4b22fbe51263ebdbfdec5e566306798115dfaa96e2f', 'facda1af501b909a64e33e8740edf8479bf6c49d9cc832a02f66264893ef0d53', 'd0bff10e10098707c06cc4ed0007559e2f8f839a5ddd770d901d35368c17cc79', '7b932b0354115553e34dd149dc59c5111e2998062d55a07384a13b4f531d5c94', '9532867a2bb2c1b932c5cd38a2178bcf0a930bb7276b2ae1044d78ca29f24e22', 'be74a873092106ee2f25862b2847422a0dbca6432f11c4164957ab38a3c1f4ad', '8e90d91bba8a2bc7be84498d54731efdb141a180df30a5216ee6f4f1c5ac6f82', 'c0fa89ea37d2cfd413941d3d96c795401885db2762c931eb8c50c38f57dd97b6', '390fe3c6bb55cfbe7edccf1e3837c2c7e0ca99d2afb44f22bb796b6992dbbde1', 'e61ee967b9034e8c75ae55fc325db5055e48281b24dcc0c0bd64e01935ba1f5d', 'ca232166038b22bc028f05cfdfa327d5c244f0236213810364f78648d39d36e2', 'e9eb88ff15b47db26645f42c1dbd1cd6b5e9f2a7eff8126792a8db9ee4173c65', '176832c8c694990ff4a45b5a16e47218fb082a74a5d59e5bee99ae4b1503e197', '9919c8ac6bcc21b38a772d6eb69c61bdcbe36f19967829e1f11fea672a8c5b93', 'dbb5782f710946b3faddfb37770ad23d90a86c3765acf942e2ebcbcb37f4d7d1', 'b06bb699e145811e16df1c8b59444d977094a5e2d4acce6a2bb03a50ba80c97b', '7e86b85004c1f57a8468b5b26254c5508d1672f95aa0adb09a979d45b8351903'
    ], 
    str("2015-20"): [
        'ff090f8169d4c5897cefdc96784ed196fae3930f0021a352ae3cb0c59b199c5a', '20dc5841ef47e7beb72b7739d24c59c69bb35108c06bd46ae5a84f552a928f3e', '2180a9965aee9f94da9a43b192d4e27bece32e55aefd4b345409bbcd112afbe6', '84c12ec847687d221767a09f1b5ff7659cf1a17e7b2c971700aee3edf94d7894', '1f40e29bca64adbd5ba565fecd2084184829d4c4b8eed3fb488925d4943ff240', 'ac2daaa58cf6405418842b103dc8074bb94e54cf15f3d02697a9ba9e140009a8', 'baaad2b363180c669598e9876b2c62d75cda148178e999555195452fd93d09bd', '29e9e01c4264cfbae58917f9bcd4d8ac9aee0de0a4fecc1f03746c565396cf3b', '21af9aeb659a15a2e9e4699f7b3a69d6d8bbef98eb4e8273301d192003dfd11e', 'd3c9bc1712a466ca23a355d80751c33e562136ecb3d5a20ee55c3ad5b581d3d3', 'bcc24af0ab59fd3ae433c10600f27d0d15e8dab2733a4c37d0d8f1d6e03bc067', '5930d72f357d9b9f2f6a3716bafc5bb6d1160dec3c7b249e00f144c1337e61be', '2297da146c3b3fd88683ff49995ceadd14f112622169cbcabef5978c81515398', '603eb415f08592485a3fda4213196665c8014f02f16b18cabb31a89af2a9924a', 'f7ad68d1fe0e335e58fc45afbdfb49e3f5b62836d5ccc0c579a145052aec9cd7', '0842a2922084d042928b7459717031818165853df759205f6ed58716c42d192d', '3438cdfaa603446ce22411a5478b1cc80e3792894dccba37ab034740afbbef4c', '63a4de8f86b47874b09121dec0c58bab8c3c3d2ca3ff1a0c0475ac25e04fdb6b', '43ace4674f842663f73d16484be103bc974f2391fbef11fea182d955e8987693'
    ]
}

In [13]:
combined_dict = {}

for key in dict_id_ori.keys():
    if key in dict_id_anon:
        combined_list = [(int(val1), val2) for val1, val2 in zip(dict_id_ori[key], dict_id_anon[key])]
        combined_dict[key] = combined_list

print(combined_dict)

{'2015-10': [(98, '848d371053f5c2eed1c84b443e4e3a6b9bb50ee3a949cd80ba50b8495d6f3d6d'), (18, 'c83ec907476a3101632c878366027ed03068d75db8accbb50675a818bae944a6'), (63, '0f27aa12029fd459c6fe3195eb801ca37acda08753ebdc9ca5a2ab7c1e791706'), (107, '3fcd9e6d2295cb93c852b9af95f8fb0fb7dc242b9557a0df45af33ecf9107c56'), (66, 'c60a6e26f74c9be0cd8b3f5fcdf8e20ae2e4c2efd20f3ec78b6b9b782665f201'), (5, '64073b3845fed8476608b0ab5a9ab52c5869a6cdbaad5c5f0faa93467d0ef6ba'), (78, 'a5e1589dc872f333508a84f551ec37aa3b701080fb47dc3ce1eb9c7442bd0599'), (32, '386d7405944c35803554176ec361cdd789920d1b1d538b759438f96acc9f58f6'), (30, '7741a739b8d6bedb5ad20c247f16f4b3842f75b5e755130f6e134e5edac9036c'), (73, '779150f54f4d27efec1b1a0fe1a84ad8f6502b8f8e1fe145d4fc8c614dc9364b'), (29, 'ec39639428c541c343da55fd9763fac38251e53316bc2926e92dd16ba85d196e'), (77, '79d1f4d0b438aa92efa4d60b0f12bac6d65d3632f4da59b25381376391726933'), (16, '4f7c38079e19d8622303e7891d4d4e34a5a3ecefa556676af4b28f1ea4ed4284'), (68, 'e77832d6d6260526c90

In [14]:
attack_VinAnonyme_701_json = {}
    
for key, value in combined_dict.items():
    inner_dict = {}
    for inner_key, inner_value in value:
        inner_dict[inner_key] = [inner_value]
    
    for inner_key, inner_value in value:
        if inner_key in attack_VinAnonyme_701_json:
            attack_VinAnonyme_701_json[inner_key][key] = [inner_value]
        else:
            attack_VinAnonyme_701_json[inner_key] = {key: [inner_value]}
            
print(attack_VinAnonyme_701_json)
print(type(inner_value))

{98: {'2015-10': ['848d371053f5c2eed1c84b443e4e3a6b9bb50ee3a949cd80ba50b8495d6f3d6d'], '2015-11': ['774dd7515528cf048a4043c319f97c16f2441628c7feacde3d980e3bc2b3d268']}, 18: {'2015-10': ['c83ec907476a3101632c878366027ed03068d75db8accbb50675a818bae944a6'], '2015-12': ['4bd27d1cdae44725f31bf58983ba7de24cd840a1dbbdc24277c2b0cd6362007c'], '2015-13': ['11dfaeae41371d9291fd18bfe7788f061658be5a96c8c89f4c8de5066b34dc61'], '2015-15': ['81ed1e7a9a6ca986070ac9a7588b5aed55f2440aa56a39acee587eeea29d32ae'], '2015-16': ['d59afc715c3daa01fcda17873436bc29527dc94d5d6e727abc70e9be7310f0f9'], '2015-17': ['b35e3e6e83f57d3b10ad69d1728621e9e5957e73d403ffcf3b7b0a822f7de17a'], '2015-18': ['dcba5f472e970004ef09495f6bcd528518c2cc0735a80ab226cd85f9dfb12759'], '2015-19': ['8ffa3ad8fe8ee82533caf4b22fbe51263ebdbfdec5e566306798115dfaa96e2f'], '2015-20': ['2180a9965aee9f94da9a43b192d4e27bece32e55aefd4b345409bbcd112afbe6']}, 63: {'2015-10': ['0f27aa12029fd459c6fe3195eb801ca37acda08753ebdc9ca5a2ab7c1e791706'], '2015-11':

In [15]:
jsonData = json.dumps(attack_VinAnonyme_701_json, indent=4)
print(jsonData)

{
    "98": {
        "2015-10": [
            "848d371053f5c2eed1c84b443e4e3a6b9bb50ee3a949cd80ba50b8495d6f3d6d"
        ],
        "2015-11": [
            "774dd7515528cf048a4043c319f97c16f2441628c7feacde3d980e3bc2b3d268"
        ]
    },
    "18": {
        "2015-10": [
            "c83ec907476a3101632c878366027ed03068d75db8accbb50675a818bae944a6"
        ],
        "2015-12": [
            "4bd27d1cdae44725f31bf58983ba7de24cd840a1dbbdc24277c2b0cd6362007c"
        ],
        "2015-13": [
            "11dfaeae41371d9291fd18bfe7788f061658be5a96c8c89f4c8de5066b34dc61"
        ],
        "2015-15": [
            "81ed1e7a9a6ca986070ac9a7588b5aed55f2440aa56a39acee587eeea29d32ae"
        ],
        "2015-16": [
            "d59afc715c3daa01fcda17873436bc29527dc94d5d6e727abc70e9be7310f0f9"
        ],
        "2015-17": [
            "b35e3e6e83f57d3b10ad69d1728621e9e5957e73d403ffcf3b7b0a822f7de17a"
        ],
        "2015-18": [
            "dcba5f472e970004ef09495f6bcd528518c2cc0735a80a